# 9장 향후 판매량 예측 경진대회 환경 세팅된 노트북 양식

In [ ]:
import pandas as pd
sales_train = pd.read_csv('')

In [ ]:
#데이터 합치기 381
train = sales_train.merge(shops, on='shop_id', how='left')
train = train.merge(items, on='item_id', how='left')
train = train.merge(item_categories, on='item_category_id', how='left')
#train.head()

In [ ]:
#요약표 383

# 데이터 시각화 384
# 일 판매량
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
#sns.boxplot(data=train, y='item_cnt_day')

# 판매가격
#sns.boxplot(data=train, y='item_price')

# 그룹화
group = train.groupby('date_block_num').agg({'item_cnt_day':'sum'})
group.reset_index()


mpl.rc('font', size=13)
figure, ax = plt.subplots()
figure.set_size_inches(11, 5)
# 월별판매량
group_month_sum = train.groupby('date_block_num').agg({'item_cnt_day':'sum'})
group_month_sum = group_month_sum.reset_index()

sns.barplot(data=group_month_sum, x='date_block_num', y='item_cnt_day')
ax.set(title='Distribution of monthly item counts by data block number', xlabel='Date block number', ylabel='Monthly item counts')

# 상품별 판매량
figure, ax = plt.subplots()
figure.set_size_inches(11,5)
group_cat_sum = train.groupby('item_category_id').agg({'item_cnt_day':'sum'})
group_cat_sum = group_cat_sum.reset_index()
group_cat_sum = group_cat_sum[group_cat_sum['item_cnt_day'] > 10000]

sns.barplot(data=group_cat_sum, x='item_category_id', y='item_cnt_day')
ax.set(title='Distribution of monthly item counts by item category id',
       xlabel='Date block number', ylabel='total item counts')

# 상점별 판매량
figure, ax = plt.subplots()
figure.set_size_inches(11,5)
group_shop_sum = train.groupby('shop_id').agg({'item_cnt_day':'sum'})
group_shop_sum = group_shop_sum.reset_index()
group_shop_sum = group_shop_sum[group_shop_sum['item_cnt_day'] > 10000]

sns.barplot(data=group_shop_sum, x='shop_id', y='item_cnt_day')
ax.set(title='Distribution of total item counts by shop id',
       xlabel='shop ID', ylabel='total item counts')
ax.tick_params(axis='x', labelrotation=90)


In [ ]:
shops = shops.rename(columns={'shop_name':'상점명', 'shop_id':'상점ID'})
#shops.head()
items = items.rename(columns={'item_name':'상품명', 'item_id':'상품ID', 'item_category_id':'상품분류ID'})
#items.head()
item_categories = item_categories.rename(columns={'item_category_name':'상품분류명', 'item_category_id':'상품분류ID'})
#item_categories.head()
test = test.rename(columns={'shop_id':'상점ID', 'item_id':'상품ID'})
#test.head()

In [ ]:
# 작은 데이터 타입으로 변경하기 396
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print('{:.1f}% 압축 됨'.format(100 * (start_mem - end_mem) / start_mem))
        
    return df


all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

In [ ]:
# 데이터 조합 398 꼭 앞 글을 읽어주세요.
from itertools import product

train=[]
for i in sales_train['월ID'].unique(): # 하나씩 추출
    all_shop = sales_train.loc[sales_train['월ID'] == i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID'] == i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))

idx_features = ['월ID','상점ID','상품ID']
train = pd.DataFrame(np.vstack(train), columns=idx_features)

train

In [ ]:
# 항목 월간 판매량 추가하기 399 
group = sales_train.groupby(idx_features).agg({'판매량': 'sum'})
group = group.reset_index()
group = group.rename(columns={'판매량':'월간 판매량'})
group

In [ ]:
# 2025년 10월 데이터 만들기
test['월ID'] = 34

#데이터 붙이기
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=idx_features
                    )

#Nan -> 0
all_data = all_data.fillna(0)
#all_data.head()

In [ ]:
#최종 403

all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')
all_data = downcast(all_data)
del shops, items, item_categories
gc.collect()

In [ ]:
all_data = all_data.drop(['상점명','상품명','상품분류명'], axis=1)